In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu  = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'
ed   = './data/mimic-iv-2.2-parquet/ed/'

In [2]:
labevents     = pd.read_parquet(hosp+'labevents.parquet')
d_labitems    = pd.read_parquet(hosp+'d_labitems.parquet')

patients      = pd.read_parquet(hosp+'patients.parquet')
admissions    = pd.read_parquet(hosp+'admissions.parquet')

diagnoses_icd = pd.read_parquet(hosp+'diagnoses_icd.parquet')
microbiology  = pd.read_parquet(hosp+'microbiologyevents.parquet')
prescriptions = pd.read_parquet(hosp+'prescriptions.parquet')

In [3]:
chartevents     = pd.read_parquet(icu+'chartevents.parquet')
d_items         = pd.read_parquet(icu+'d_items.parquet')
inputevents     = pd.read_parquet(icu+'inputevents.parquet')
outputevents    = pd.read_parquet(icu+'outputevents.parquet')
procedureevents = pd.read_parquet(icu+'procedureevents.parquet')
icustays        = pd.read_parquet(icu+'icustays.parquet')

In [130]:
vitalsign = pd.read_parquet(ed+'vitalsign.parquet')

In [38]:
outputevents    = pd.read_parquet(icu+'outputevents.parquet')

In [4]:
# to_datetime
chartevents['charttime'] = pd.to_datetime(chartevents['charttime'])

In [17]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
vitals = pd.read_csv('./data/vitals/vitals.csv')

# Demographic

### Gender

In [114]:
patients_gender = patients[['subject_id','gender']]
patients_gender.loc[patients_gender.gender == 'F' , 'gender'] = 1
patients_gender.loc[patients_gender.gender == 'M' , 'gender'] = 0

In [115]:
patients_gender

subject_id gender
0         10000032      1
1         10000048      1
2         10000068      1
3         10000084      0
4         10000102      1
...            ...    ...
299707    19999828      1
299708    19999829      1
299709    19999840      0
299710    19999914      1
299711    19999987      1

[299712 rows x 2 columns]

### Age

In [116]:
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

In [117]:
icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
patients_age = patients[['subject_id','anchor_age','anchor_year']]

icustays_age = pd.merge(icustays_intime, patients_age, on = 'subject_id', how = 'left')

In [118]:
icustays_age['anchor_age_delta'] = pd.to_timedelta(icustays_age['anchor_age']*365.25, unit='D')
icustays_age['anchor_year'] = pd.to_datetime(icustays_age['anchor_year'],format="%Y")

icustays_age['delta'] = icustays_age['intime'] - icustays_age['anchor_year']
icustays_age['age'] = ((icustays_age['anchor_age_delta'] + icustays_age['delta'])/365.25).dt.days

icustays_age = icustays_age[['subject_id','hadm_id','stay_id','age']]

In [119]:
icustays_age

subject_id   hadm_id   stay_id  age
0        10000032  29079034  39553978   52
1        10000980  26913865  39765666   76
2        10001217  24597018  37067082   55
3        10001217  27703517  34592300   55
4        10001725  25563031  31205490   46
...           ...       ...       ...  ...
73176    19999442  26785317  32336619   43
73177    19999625  25304202  31070865   82
73178    19999828  25744818  36075953   48
73179    19999840  21033226  38978960   58
73180    19999987  23865745  36195440   57

[73181 rows x 4 columns]

### Race

In [57]:
admissions_race = admissions[['subject_id','race']].copy()

In [58]:
admissions_race['race'].replace(['ASIAN - ASIAN INDIAN', 'ASIAN - CHINESE','ASIAN - KOREAN', 'ASIAN - SOUTH EAST ASIAN'],'ASIAN',inplace=True)
admissions_race['race'].replace(['BLACK/AFRICAN AMERICAN','BLACK/AFRICAN','BLACK/CAPE VERDEAN','BLACK/CARIBBEAN ISLAND'],'BLACK',inplace=True)
admissions_race['race'].replace(['HISPANIC/LATINO - CENTRAL AMERICAN','HISPANIC/LATINO - COLUMBIAN','HISPANIC/LATINO - CUBAN','HISPANIC/LATINO - DOMINICAN','HISPANIC/LATINO - GUATEMALAN','HISPANIC/LATINO - HONDURAN','HISPANIC/LATINO - MEXICAN',
                             'HISPANIC/LATINO - PUERTO RICAN','HISPANIC/LATINO - SALVADORAN','PORTUGUESE','SOUTH AMERICAN'],'HISPANIC OR LATINO',inplace=True)
admissions_race['race'].replace(['NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER'],'PACIFIC ISLANDER',inplace=True)
admissions_race['race'].replace(['UNABLE TO OBTAIN','PATIENT DECLINED TO ANSWER'],'UNKNOWN',inplace=True)
admissions_race['race'].replace(['WHITE - BRAZILIAN','WHITE - EASTERN EUROPEAN','WHITE - OTHER EUROPEAN','WHITE - RUSSIAN'],'WHITE',inplace=True)

admissions_race = admissions_race.drop_duplicates()

In [61]:
multiple = admissions_race.subject_id.value_counts().loc[lambda x : x > 1].to_frame()
multiple.reset_index(inplace = True)
multiple = multiple.subject_id.unique()

hosp_race_multiple = admissions_race[admissions_race['subject_id'].isin(multiple)]

sol = []
multi = []
for i in hosp_race_multiple.subject_id.unique() :
    tmp = hosp_race_multiple[hosp_race_multiple['subject_id'] == i]
    if (tmp['race'] == 'UNKNOWN').any() :
        tmp = tmp[tmp['race'] != 'UNKNOWN']
    if len(tmp) <2 :
        sol.append(tmp)
    else : 
        multi.append(tmp)
sol = pd.concat(sol)
multi = pd.concat(multi)

multi['race'] = 'MULTIPLE RACE/ETHNICITY'
multi.drop_duplicates(inplace=True)

admissions_race = admissions_race[~admissions_race['subject_id'].isin(sol.subject_id.unique())]
admissions_race = admissions_race[~admissions_race['subject_id'].isin(multi.subject_id.unique())]

admissions_race = pd.concat([admissions_race,sol,multi])

In [62]:
admissions_race.race.unique()

array(['WHITE', 'OTHER', 'BLACK', 'UNKNOWN', 'HISPANIC OR LATINO',
       'ASIAN', 'PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKA NATIVE',
       'MULTIPLE RACE/ETHNICITY'], dtype=object)

## Height

In [125]:
# Height
chartevents[chartevents['itemid'].isin([226707])]

subject_id   hadm_id   stay_id  caregiver_id           charttime  \
157          10000032  29079034  39553978       88981.0 2180-07-23 12:36:00   
3718         10001725  25563031  31205490       22280.0 2110-04-11 15:52:00   
4644         10001884  26184834  37510196        3512.0 2131-01-11 04:20:00   
20745        10002013  23581541  39060235       96730.0 2160-05-18 10:00:00   
40604        10002428  20321825  34807493       62692.0 2156-04-30 20:36:00   
...               ...       ...       ...           ...                 ...   
313613045    19999287  20175828  35165301       34445.0 2197-08-03 20:58:00   
313616169    19999287  22997012  37692584       11933.0 2197-07-26 03:31:00   
313621571    19999297  21439025  37364566       30382.0 2162-08-16 05:48:00   
313626294    19999442  26785317  32336619       29154.0 2148-11-19 14:23:00   
313641209    19999840  21033226  38978960       82641.0 2164-09-12 09:26:00   

                     storetime  itemid value  valuenum valueuom  warning  
157        2180-07-23 14:45:00  226707    60      60.0     Inch      0.0  
3718       2110-04-11 16:01:00  226707    62      62.0     Inch      0.0  
4644       2131-01-13 14:43:00  226707    62      62.0     Inch      0.0  
20745      2160-05-18 12:05:00  226707    62      62.0     Inch      0.0  
40604      2156-05-02 16:37:00  226707    59      59.0     Inch      0.0  
...                        ...     ...   ...       ...      ...      ...  
313613045  2197-08-06 15:12:00  226707    65      65.0     Inch      0.0  
313616169  2197-07-27 14:42:00  226707    64      64.0     Inch      0.0  
313621571  2162-08-16 11:48:00  226707    65      65.0     Inch      0.0  
313626294  2148-11-20 16:34:00  226707    76      76.0     Inch      0.0  
313641209  2164-09-12 09:47:00  226707    70      70.0     Inch      0.0  

[33707 rows x 11 columns]

In [126]:
# Height(cm)
chartevents[chartevents['itemid'].isin([226730])]

subject_id   hadm_id   stay_id  caregiver_id           charttime  \
158          10000032  29079034  39553978       88981.0 2180-07-23 12:36:00   
3719         10001725  25563031  31205490       22280.0 2110-04-11 15:52:00   
4645         10001884  26184834  37510196        3512.0 2131-01-11 04:20:00   
20746        10002013  23581541  39060235       96730.0 2160-05-18 10:00:00   
40605        10002428  20321825  34807493       62692.0 2156-04-30 20:36:00   
...               ...       ...       ...           ...                 ...   
313613046    19999287  20175828  35165301       34445.0 2197-08-03 20:58:00   
313616170    19999287  22997012  37692584       11933.0 2197-07-26 03:31:00   
313621572    19999297  21439025  37364566       30382.0 2162-08-16 05:48:00   
313626295    19999442  26785317  32336619       29154.0 2148-11-19 14:23:00   
313641210    19999840  21033226  38978960       82641.0 2164-09-12 09:26:00   

                     storetime  itemid value  valuenum valueuom  warning  
158        2180-07-23 14:45:00  226730   152     152.0       cm      0.0  
3719       2110-04-11 16:01:00  226730   157     157.0       cm      0.0  
4645       2131-01-13 14:43:00  226730   157     157.0       cm      0.0  
20746      2160-05-18 12:05:00  226730   157     157.0       cm      0.0  
40605      2156-05-02 16:37:00  226730   150     150.0       cm      0.0  
...                        ...     ...   ...       ...      ...      ...  
313613046  2197-08-06 15:12:00  226730   165     165.0       cm      0.0  
313616170  2197-07-27 14:42:00  226730   163     163.0       cm      0.0  
313621572  2162-08-16 11:48:00  226730   165     165.0       cm      0.0  
313626295  2148-11-20 16:34:00  226730   193     193.0       cm      0.0  
313641210  2164-09-12 09:47:00  226730   178     178.0       cm      0.0  

[33707 rows x 11 columns]

## Weight

In [124]:
#Admission Weight(Kg)
chartevents[chartevents['itemid'].isin([226512])]

subject_id   hadm_id   stay_id  caregiver_id           charttime  \
156          10000032  29079034  39553978       88981.0 2180-07-23 12:36:00   
488          10000980  26913865  39765666       26402.0 2189-06-27 07:40:00   
1663         10001217  24597018  37067082       84976.0 2157-11-20 19:17:00   
2881         10001217  27703517  34592300       51676.0 2157-12-19 15:42:00   
3717         10001725  25563031  31205490       22280.0 2110-04-11 15:52:00   
...               ...       ...       ...           ...                 ...   
313626293    19999442  26785317  32336619       29154.0 2148-11-19 14:23:00   
313633288    19999625  25304202  31070865       84925.0 2139-10-10 18:06:00   
313633978    19999828  25744818  36075953       44654.0 2149-01-08 16:45:00   
313641208    19999840  21033226  38978960       82641.0 2164-09-12 09:26:00   
313643046    19999987  23865745  36195440       49819.0 2145-11-02 21:39:00   

                     storetime  itemid  value  valuenum valueuom  warning  
156        2180-07-23 14:45:00  226512   39.4      39.4       kg      0.0  
488        2189-06-27 09:02:00  226512   76.2      76.2       kg      0.0  
1663       2157-11-20 19:32:00  226512   71.2      71.2       kg      0.0  
2881       2157-12-19 17:26:00  226512   74.8      74.8       kg      0.0  
3717       2110-04-11 16:01:00  226512   72.2      72.2       kg      0.0  
...                        ...     ...    ...       ...      ...      ...  
313626293  2148-11-20 16:34:00  226512  107.5     107.5       kg      0.0  
313633288  2139-10-10 19:58:00  226512   50.5      50.5       kg      0.0  
313633978  2149-01-08 18:15:00  226512   67.9      67.9       kg      0.0  
313641208  2164-09-12 09:47:00  226512   77.5      77.5       kg      0.0  
313643046  2145-11-03 15:10:00  226512     94      94.0       kg      0.0  

[67826 rows x 11 columns]

In [123]:
#Admission Weight(lbs.)
chartevents[chartevents['itemid'].isin([226531])]

subject_id   hadm_id   stay_id  caregiver_id           charttime  \
242          10000032  29079034  39553978       88981.0 2180-07-23 14:22:00   
264          10000032  29079034  39553978       88981.0 2180-07-23 14:44:00   
482          10000980  26913865  39765666       11933.0 2189-06-27 18:02:00   
485          10000980  26913865  39765666       11933.0 2189-06-27 20:13:00   
525          10000980  26913865  39765666       36518.0 2189-06-27 08:27:00   
...               ...       ...       ...           ...                 ...   
313632793    19999442  26785317  32336619       96670.0 2148-11-24 07:59:00   
313633287    19999625  25304202  31070865       44373.0 2139-10-11 10:02:00   
313643257    19999987  23865745  36195440       49819.0 2145-11-03 09:01:00   
313643342    19999987  23865745  36195440       49819.0 2145-11-03 11:59:00   
313643582    19999987  23865745  36195440       49819.0 2145-11-03 15:10:00   

                     storetime  itemid  value  valuenum valueuom  warning  
242        2180-07-23 14:23:00  226531   86.7      86.7     None      0.0  
264        2180-07-23 14:45:00  226531   86.7      86.7     None      0.0  
482        2189-06-27 18:02:00  226531  167.6     167.6     None      0.0  
485        2189-06-27 20:13:00  226531  167.6     167.6     None      0.0  
525        2189-06-27 08:27:00  226531      0       0.0     None      0.0  
...                        ...     ...    ...       ...      ...      ...  
313632793  2148-11-24 07:59:00  226531  236.5     236.5     None      0.0  
313633287  2139-10-11 10:02:00  226531  111.1     111.1     None      0.0  
313643257  2145-11-03 09:01:00  226531    132     132.0     None      0.0  
313643342  2145-11-03 11:59:00  226531    132     132.0     None      0.0  
313643582  2145-11-03 15:10:00  226531  206.8     206.8     None      0.0  

[119251 rows x 11 columns]

# Lab values

In [72]:
for i,idx in tqdm(enumerate(labvalues.abbreviation)):
    print("[%i/%i] Processing %s..."%(i+1,len(labvalues),labvalues.iloc[i].labvalue))
    globals()['chartevents_{}'.format(idx)] = chartevents[chartevents['itemid'].isin([labvalues.iloc[i].itemid_icu])][['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

    if labvalues.iloc[i].lb_cond == 'ge' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] >= labvalues.iloc[i].lb]
    elif labvalues.iloc[i].lb_cond == 'gt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] > labvalues.iloc[i].lb]

    if labvalues.iloc[i].ub_cond == 'le' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] <= labvalues.iloc[i].ub]
    elif labvalues.iloc[i].ub_cond == 'lt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] < labvalues.iloc[i].ub]

    globals()['chartevents_{}'.format(idx)].sort_values(by=['subject_id','charttime'],ascending=True,inplace=True)
    globals()['chartevents_{}'.format(idx)].reset_index(inplace=True, drop=True)

    if not os.path.isfile('./data/labvalues/chartevents_%s.parquet'%idx):
        globals()['chartevents_{}'.format(idx)].to_parquet('./data/labvalues/chartevents_%s.parquet'%idx)

0it [00:00, ?it/s]

[1/28] Processing Albumin...
[2/28] Processing Alkaline Phosphate...
[3/28] Processing Anion Gap...
[4/28] Processing Blood urea nitrogen...
[5/28] Processing Calcium non-ionized...
[6/28] Processing Creatine Kinase...
[7/28] Processing Direct Bilirubin...
[8/28] Processing Glucose...
[9/28] Processing Hematocrit...
[10/28] Processing International Normalized Ratio...
[11/28] Processing PH...
[12/28] Processing Phosphorous...
[13/28] Processing Platelet Count...
[14/28] Processing Serum chloride...
[15/28] Processing Serum creatinine...
[16/28] Processing Serum Sodium...
[17/28] Processing Serum Potassium...
[18/28] Processing Total Bilirubin...
[19/28] Processing White blood cell count...
[20/28] Processing Serum glucose...
[21/28] Processing Magnesium...
[22/28] Processing Ionized calcium...
[23/28] Processing Serum HCO3...
[24/28] Processing AST...
[25/28] Processing ALT...
[26/28] Processing PTT...
[27/28] Processing Arterial Base Excess...
[28/28] Processing Lactic Acid...


In [6]:
for i,idx in tqdm(enumerate(labvalues.abbreviation)):
    try :
        print("[%i/%i] Processing %s..."%(i+1,len(labvalues),labvalues.iloc[i].labvalue))
        itemid = labvalues.iloc[i].itemid_hosp
        if isinstance(itemid, str):
            itemid = itemid.replace(" ","")
            itemid = itemid.split(",")
        if isinstance(itemid, list):
            itemid = [int(x) for x in itemid]
        else : itemid = [int(itemid)]
        globals()['labevents_{}'.format(idx)] = labevents[labevents['itemid'].isin(itemid)][['subject_id','hadm_id','charttime','itemid','valuenum']]

        if labvalues.iloc[i].lb_cond == 'ge' : 
            globals()['labevents_{}'.format(idx)] = globals()['labevents_{}'.format(idx)][globals()['labevents_{}'.format(idx)]['valuenum'] >= labvalues.iloc[i].lb]
        elif labvalues.iloc[i].lb_cond == 'gt' : 
            globals()['labevents_{}'.format(idx)] = globals()['labevents_{}'.format(idx)][globals()['labevents_{}'.format(idx)]['valuenum'] > labvalues.iloc[i].lb]

        if labvalues.iloc[i].ub_cond == 'le' : 
            globals()['labevents_{}'.format(idx)] = globals()['labevents_{}'.format(idx)][globals()['labevents_{}'.format(idx)]['valuenum'] <= labvalues.iloc[i].ub]
        elif labvalues.iloc[i].ub_cond == 'lt' : 
            globals()['labevents_{}'.format(idx)] = globals()['labevents_{}'.format(idx)][globals()['labevents_{}'.format(idx)]['valuenum'] < labvalues.iloc[i].ub]

        globals()['labevents_{}'.format(idx)].sort_values(by=['subject_id','charttime'],ascending=True,inplace=True)
        globals()['labevents_{}'.format(idx)].reset_index(inplace=True, drop=True)

        if not os.path.isfile('./data/labvalues/labevents_%s.parquet'%idx):
            globals()['labevents_{}'.format(idx)].to_parquet('./data/labvalues/labevents_%s.parquet'%idx)
    except: ValueError

0it [00:00, ?it/s]

[1/28] Processing Albumin...
[2/28] Processing Alkaline Phosphate...
[3/28] Processing Anion Gap...
[4/28] Processing Blood urea nitrogen...
[5/28] Processing Calcium non-ionized...
[6/28] Processing Creatine Kinase...
[7/28] Processing Direct Bilirubin...
[8/28] Processing Glucose...
[9/28] Processing Hematocrit...
[10/28] Processing International Normalized Ratio...
[11/28] Processing PH...
[12/28] Processing Phosphorous...
[13/28] Processing Platelet Count...
[14/28] Processing Serum chloride...
[15/28] Processing Serum creatinine...
[16/28] Processing Serum Sodium...
[17/28] Processing Serum Potassium...
[18/28] Processing Total Bilirubin...
[19/28] Processing White blood cell count...
[20/28] Processing Serum glucose...
[21/28] Processing Magnesium...
[22/28] Processing Ionized calcium...
[23/28] Processing Serum HCO3...
[24/28] Processing AST...
[25/28] Processing ALT...
[26/28] Processing PTT...
[27/28] Processing Arterial Base Excess...
[28/28] Processing Lactic Acid...


# Vitals

In [18]:
for i,idx in tqdm(enumerate(vitals.abbreviation)):
    print("[%i/%i] Processing %s..."%(i+1,len(vitals),vitals.iloc[i].labvalue))
    itemid = vitals.iloc[i].itemid_icu
    if isinstance(itemid, str):
        itemid = itemid.replace(" ","")
        itemid = itemid.split(",")
    if isinstance(itemid, list):
        itemid = [int(x) for x in itemid]
    else : itemid = [int(itemid)]
    globals()['chartevents_{}'.format(idx)] = chartevents[chartevents['itemid'].isin(itemid)][['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

    if vitals.iloc[i].lb_cond == 'ge' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] >= vitals.iloc[i].lb]
    elif vitals.iloc[i].lb_cond == 'gt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] > vitals.iloc[i].lb]

    if vitals.iloc[i].ub_cond == 'le' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] <= vitals.iloc[i].ub]
    elif vitals.iloc[i].ub_cond == 'lt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] < vitals.iloc[i].ub]

    globals()['chartevents_{}'.format(idx)].sort_values(by=['subject_id','charttime'],ascending=True,inplace=True)
    globals()['chartevents_{}'.format(idx)].reset_index(inplace=True, drop=True)

    if not os.path.isfile('./data/vitals/chartevents_%s.parquet'%idx):
        globals()['chartevents_{}'.format(idx)].to_parquet('./data/vitals/chartevents_%s.parquet'%idx)

0it [00:00, ?it/s]

[1/13] Processing Arterial O2 pressure...
[2/13] Processing Arterial CO2 Pressure...
[3/13] Processing Inspired O2 Fraction...
[4/13] Processing Arterial O2 Saturation...
[5/13] Processing Respiratory Rate...
[6/13] Processing Arterial Blood Pressure systolic...
[7/13] Processing Arterial Blood Pressure diastolic...
[8/13] Processing Non Invasive Blood Pressure systolic...
[9/13] Processing Non Invasive Blood Pressure diastolic...
[10/13] Processing Temperature Celsius...
[11/13] Processing Temperature Fahrenheit...
[12/13] Processing Heart Rate...
[13/13] Processing Central Venous Pressure...


## Temp

In [133]:
chartevents_tempF['valuenum2'] = (chartevents_tempF['valuenum']-32)*5/9
chartevents_tempF.drop('valuenum',axis=1,inplace=True)
chartevents_tempF.rename(columns={'valuenum2':'valuenum'},inplace=True)
chartevents_tempF['valuenum'] = chartevents_tempF['valuenum'].round(1)

chartevents_temp = pd.concat([chartevents_tempC,chartevents_tempF])
chartevents_temp.sort_values(by=['subject_id','hadm_id','stay_id','charttime'],inplace=True)
chartevents_temp.to_parquet('./data/vitals/chartevents_temp.parquet')

# Urine Output

In [58]:
outputevents_uo = outputevents[outputevents['itemid'].isin([
    226557, 226558, #Ureteral Stent, 요관스텐트
    226559, #Foley, 도뇨관(소변줄)
    226560, 226561, 226563, 226564, 226565, 226566, 226567, 226584, 226627, 226631, 226632
])]

guirrigant_input = outputevents[outputevents['itemid'].isin([227488])]
guirrigant_output = outputevents[outputevents['itemid'].isin([227489])]

guirrigant = pd.merge(guirrigant_input,guirrigant_output[['subject_id','hadm_id','stay_id','charttime','value']],on=['subject_id','hadm_id','stay_id','charttime'])
guirrigant['value'] =  guirrigant['value_y'] - guirrigant['value_x']
guirrigant = guirrigant[guirrigant['value'] >= 0]
guirrigant = guirrigant[['subject_id','hadm_id','stay_id','charttime','itemid','value']]

outputevents_uo = outputevents_uo[['subject_id','hadm_id','stay_id','charttime','itemid','value']]
outputevents_uo = pd.concat([outputevents_uo,guirrigant])

outputevents_uo.sort_values(['subject_id','hadm_id','stay_id','charttime'],inplace=True)
outputevents_uo.reset_index(inplace=True,drop=True)

outputevents_uo.to_parquet('./data/others/outputevents_uo.parquet')

# Baseline SCr

In [70]:
def baseline_sCr(chartevents_SCr,icustays,labevents) :
    icu_sCr_subject_id = chartevents_SCr['subject_id'].unique()
    icu_sCr_stay_id = chartevents_SCr['stay_id'].unique()

    icu_icustays_sCr = icustays[icustays['stay_id'].isin(icu_sCr_stay_id)]
    icu_icustays_sCr['intime'] = pd.to_datetime(icu_icustays_sCr['intime'])

    hosp_labevents_sCr = labevents[labevents['itemid'].isin([
    50912, # Creatinine, Blood, Chemistry
    52024, # Creatinine, Whole Blood, Blood, Chemistry
    52546  # Creatinine, Blood, Chemistry
    ])]

    hosp_labevents_sCr = hosp_labevents_sCr[['subject_id','hadm_id','charttime','valuenum']]
    hosp_labevents_sCr['charttime'] = pd.to_datetime(hosp_labevents_sCr['charttime'])

    baseline_sCr_list = []

    for i in tqdm(icu_sCr_subject_id) : 
        tmp_hosp_labevents_sCr = hosp_labevents_sCr[hosp_labevents_sCr['subject_id'] == i]
        for j in icu_icustays_sCr[icu_icustays_sCr['subject_id'] == i].stay_id : 
            tmp_icu_icustays_intime = icu_icustays_sCr[icu_icustays_sCr['stay_id'] == j].iloc[0]
            tmp_icu_icustays_intime_7days = tmp_icu_icustays_intime['intime'] - timedelta(days=7)
            tmp_icu_icustays_intime_1yr = tmp_icu_icustays_intime['intime'] - timedelta(days=365)

            baseline_sCr = 0
            
            tmp_hosp_labevents_sCr_list = tmp_hosp_labevents_sCr[tmp_hosp_labevents_sCr['charttime'] < tmp_icu_icustays_intime['intime']]
            tmp_hosp_labevents_sCr_list = tmp_hosp_labevents_sCr_list[tmp_hosp_labevents_sCr_list['charttime'] > tmp_icu_icustays_intime_7days]

            if not tmp_hosp_labevents_sCr_list.empty :
                baseline_sCr = tmp_hosp_labevents_sCr_list['valuenum'].min()

            else : 
                tmp_hosp_labevents_sCr_list = tmp_hosp_labevents_sCr_list[tmp_hosp_labevents_sCr_list['charttime'] < tmp_icu_icustays_intime_7days]
                tmp_hosp_labevents_sCr_list = tmp_hosp_labevents_sCr_list[tmp_hosp_labevents_sCr_list['charttime'] > tmp_icu_icustays_intime_1yr]
                baseline_sCr = tmp_hosp_labevents_sCr_list['valuenum'].median()

            baseline_sCr_list.append([i, j, baseline_sCr])

    df_baseline_sCr_list = pd.DataFrame(baseline_sCr_list, columns=['subject_id','stay_id','valuenum'])
    return df_baseline_sCr_list

In [73]:
baseline_sCr = baseline_sCr(chartevents_SCr,icustays,labevents)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_124660\3882652418.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icu_icustays_sCr['intime'] = pd.to_datetime(icu_icustays_sCr['intime'])


  0%|          | 0/49635 [00:00<?, ?it/s]

# Glasgow Coma Scale

In [74]:
chartevents_GCS_eye = chartevents[chartevents['itemid'].isin([220739])]
chartevents_GCS_verval = chartevents[chartevents['itemid'].isin([223900])]
chartevents_GCS_motor = chartevents[chartevents['itemid'].isin([223901])]

In [128]:
chartevents_GCS_eye.to_parquet('./data/others/chartevents_GCS_eye.parquet')
chartevents_GCS_verval.to_parquet('./data/others/chartevents_GCS_verval.parquet')
chartevents_GCS_motor.to_parquet('./data/others/chartevents_GCS_motor.parquet')

# Fluid/Vasopressor intake

## Fluid

In [15]:
fluid = [220949, 220950, 220952, 225158, 225159, 225161, 225828, 225797, 225799, 225823, 225825, 225827, 225830, 226089, 225941, 225943, 225944, 226361, 226363, 226364, 226375, 226377, 226452, 226453, 227533, 228140, 228141, 228142, 228341, 220955, 220967, 220968, 220953]
inputevents_fluid = inputevents[inputevents['itemid'].isin(fluid)]

In [137]:
inputevents_fluid.amountuom.unique()

array(['ml', 'L', 'nL', 'cm3', 'mm^3', 'pL'], dtype=object)

In [153]:
inputevents_fluid.loc[inputevents_fluid['amountuom']=='L', 'amount'] = inputevents_fluid['amount']*1000
inputevents_fluid.loc[inputevents_fluid['amountuom']=='nL', 'amount'] = inputevents_fluid['amount']*1e-6
inputevents_fluid.loc[inputevents_fluid['amountuom']=='mm^3', 'amount'] = inputevents_fluid['amount']*1e-3
inputevents_fluid.loc[inputevents_fluid['amountuom']=='pL', 'amount'] = inputevents_fluid['amount']*1e-9

In [176]:
from pandas import Timedelta

def calculate_drug_amounts_proportionally(inputevents_,icustays):
    rt = []
    # Select the ICU stay record
    inputevents_['starttime'] = pd.to_datetime(inputevents_['starttime'])
    inputevents_['endtime'] = pd.to_datetime(inputevents_['endtime'])
    icustays['intime'] = pd.to_datetime(icustays['intime'])
    icustays['outtime'] = pd.to_datetime(icustays['outtime'])
    
    for stay_id in tqdm(icustays.stay_id.unique()):
        icu_stay = icustays[icustays['stay_id'] == stay_id]
        tmp_id = icustays[icustays['stay_id']==stay_id][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
        # Create a date range from intime to outtime with 1-hour frequency
        time_range = pd.date_range(start=icu_stay['intime'].iloc[0], end=icu_stay['outtime'].iloc[0], freq='H')

        # Select the drug administration records for this ICU stay
        drug_records = inputevents_[inputevents_['stay_id'] == stay_id]

        # Initialize a DataFrame to store the total drug amount for each hour
        drug_amounts = pd.DataFrame(index=time_range, columns=['valuenum'])
        drug_amounts['valuenum'] = 0

        # Calculate the total drug amount for each hour
        for start, end, amount in zip(drug_records['starttime'], drug_records['endtime'], drug_records['amount']):
            # For each drug record, find the overlapping time with the hourly intervals
            overlap = time_range[(time_range >= start) & (time_range < end)]
            if len(overlap) > 0:
                # Calculate the total duration of drug administration in seconds
                total_duration = (end - start).total_seconds()
                # Calculate the duration of each overlapping hour in seconds
                for hour in overlap:
                    hour_end = hour + Timedelta(hours=1)
                    # Intersection of the drug administration period and the hour
                    intersection_start = max(start, hour)
                    intersection_end = min(end, hour_end)
                    intersection_duration = (intersection_end - intersection_start).total_seconds()
                    # Distribute the drug amount proportionally to the intersection duration
                    proportional_amount = (intersection_duration / total_duration) * amount
                    drug_amounts.loc[hour, 'valuenum'] += proportional_amount
            
        drug_amounts.reset_index(drop=False,inplace=True)
        drug_amounts.rename(columns={'index':'charttime'},inplace=True)
        drug_amounts[['subject_id','hadm_id','stay_id']] = tmp_id
        drug_amounts = drug_amounts[['subject_id','hadm_id','stay_id','charttime','valuenum']]
        rt.append(drug_amounts)
    return pd.concat(rt)

# Test the function with the same sample stay_id
drug_amounts_sample_proportional = calculate_drug_amounts_proportionally(inputevents_fluid,icustays)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_124660\4199976814.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_['starttime'] = pd.to_datetime(inputevents_['starttime'])
C:\Users\DAHS\AppData\Local\Temp\ipykernel_124660\4199976814.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_['endtime'] = pd.to_datetime(inputevents_['endtime'])


  0%|          | 0/73181 [00:00<?, ?it/s]

In [179]:
inputevents_fluid = drug_amounts_sample_proportional.copy()
inputevents_fluid.to_parquet('./data/others/inptevents_fluid.parquet')

## Vasopressor

In [100]:
inputevents_epinephrine = inputevents[inputevents['itemid'].isin([221289])]
inputevents_dopamine = inputevents[inputevents['itemid'].isin([221662])]
inputevents_dobutamine = inputevents[inputevents['itemid'].isin([221653])]
inputevents_norephinephrine = inputevents[inputevents['itemid'].isin([221906])]
inputevents_phenylephrine = inputevents[inputevents['itemid'].isin([221749, 229630, 229632])]
inputevents_vasopressin = inputevents[inputevents['itemid'].isin([222315])]

In [101]:
inputevents_norephinephrine.rateuom.unique()

array(['mcg/kg/min', 'mg/kg/min'], dtype=object)

In [ ]:
inputevents_norephinephrine.loc[inputevents_norephinephrine['rateuom']]

In [102]:
inputevents_epinephrine.rateuom.unique()

array(['mcg/kg/min'], dtype=object)

In [103]:
inputevents_phenylephrine.rateuom.unique()

array(['mcg/kg/min', 'mcg/min'], dtype=object)

In [105]:
inputevents_vasopressin.rateuom.unique()

array(['units/hour', 'units/min'], dtype=object)

In [181]:
inputevents_dopamine.patientweight.isna().sum()

0

In [182]:
inputevents_dobutamine[['amount','rate']]

amount      rate
24575    145.498015  2.503062
24603    104.201685  5.002001
24613      5.885651  2.502403
24694    248.500002  5.002819
24734     13.025211  2.501001
...             ...       ...
8947554  191.709037  4.001090
8947592  145.305547  4.000660
8969293    7.133995  5.002802
8969297    0.496658  8.010621
8969299   75.721034  5.005357

[8513 rows x 2 columns]

In [109]:
procedureevents[procedureevents['itemid'].isin([225802])]

subject_id   hadm_id   stay_id  caregiver_id            starttime  \
220       10004235  24181354  34100191       96601.0  2196-02-26 01:00:00   
621       10007818  22987108  32359580       29855.0  2146-06-26 01:00:00   
638       10007818  22987108  32359580       65154.0  2146-06-22 21:10:00   
641       10007818  22987108  32359580       65154.0  2146-06-26 16:30:00   
653       10007818  22987108  32359580       82842.0  2146-06-24 12:23:00   
...            ...       ...       ...           ...                  ...   
694893    19986880  28386154  32959861       53913.0  2185-08-06 11:20:00   
694896    19986880  28386154  32959861       76255.0  2185-08-08 11:40:00   
694899    19986880  28386154  32959861       86518.0  2185-08-04 16:10:00   
694900    19986880  28386154  32959861       86518.0  2185-08-04 17:30:00   
695055    19989783  26984195  32761676        1195.0  2130-07-13 00:00:00   

                    endtime            storetime  itemid        value  \
220     2196-02-27 16:27:00  2196-02-27 16:42:00  225802  2367.000000   
621     2146-06-26 14:21:00  2146-06-26 01:00:00  225802   801.000000   
638     2146-06-24 09:58:00  2146-06-24 10:06:00  225802  2208.000000   
641     2146-06-28 08:00:00  2146-06-28 06:39:00  225802     1.645833   
653     2146-06-26 01:11:00  2146-06-24 12:23:00  225802  2208.000000   
...                     ...                  ...     ...          ...   
694893  2185-08-08 02:28:00  2185-08-06 11:41:00  225802  2348.000000   
694896  2185-08-10 02:48:00  2185-08-08 11:40:00  225802  2348.000000   
694899  2185-08-04 17:00:00  2185-08-04 18:26:00  225802    50.000000   
694900  2185-08-06 08:38:00  2185-08-06 09:52:00  225802  2348.000000   
695055  2130-07-19 15:00:00  2130-07-19 17:35:00  225802  9540.000000   

       valueuom  ...  orderid linkorderid  ordercategoryname  \
220         min  ...  3699739     3699739           Dialysis   
621         min  ...  5520577     5520577           Dialysis   
638         min  ...  8439366     8439366           Dialysis   
641         day  ...  5677703     5677703           Dialysis   
653         min  ...  3377209     3377209           Dialysis   
...         ...  ...      ...         ...                ...   
694893      min  ...  2552802     2552802           Dialysis   
694896      min  ...  8042255     8042255           Dialysis   
694899      min  ...  4163989     4163989           Dialysis   
694900      min  ...  8782702     8782702           Dialysis   
695055      min  ...  9756350     9756350           Dialysis   

        ordercategorydescription patientweight isopenbag  continueinnextdept  \
220            ContinuousProcess         127.0         1                   0   
621            ContinuousProcess          86.2         0                   0   
638            ContinuousProcess          86.2         1                   0   
641            ContinuousProcess          86.2         0                   0   
653            ContinuousProcess          86.2         0                   0   
...                          ...           ...       ...                 ...   
694893         ContinuousProcess          57.0         0                   0   
694896         ContinuousProcess          57.0         0                   0   
694899         ContinuousProcess          57.0         1                   0   
694900         ContinuousProcess          57.0         1                   0   
695055         ContinuousProcess         101.0         1                   0   

        statusdescription  originalamount originalrate  
220       FinishedRunning          2367.0            1  
621               Stopped          2208.0            1  
638       FinishedRunning          2208.0            1  
641               Stopped             4.0            1  
653       FinishedRunning          2208.0            1  
...                   ...             ...          ...  
694893    FinishedRunning          2348.0            1  
694896    FinishedRu

In [110]:
resample_AG = pd.read_parquet('./data/resample/resample_AG.parquet')
resample_Alb = pd.read_parquet('./data/resample/resample_Alb.parquet')
